In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json

In [2]:
sensors = []
beacons = []

with open("./data/day15.txt") as f:
    while line := f.readline():
        line = line.rstrip().split(': ')
        sensor = tuple(map(lambda x: int(x[2:]), line[0].split('at ')[1].split(', ')))
        sensors.append(sensor)
        beacon = tuple(map(lambda x: int(x[2:]), line[1].split('at ')[1].split(', ')))
        beacons.append(beacon)
len(sensors), sensors[0], beacons[0]

(36, (489739, 1144461), (-46516, 554951))

In [3]:
def dist(point1, point2):
    return sum(map(lambda x: abs(x[0]-x[1]), zip(point1, point2)))

In [4]:
y_target = 2000000

points = set()
for sensor, beacon in zip(sensors, beacons):
    d = dist(sensor, beacon)
    d2 = abs(sensor[1]-y_target)
    for i in range(d-d2+1):
        points.add((sensor[0]+i, y_target))
        points.add((sensor[0]-i, y_target))
points = points.difference(set(beacons))
len(points)

5878678

In [5]:
distances = [dist(sensor, beacon) for sensor, beacon in zip(sensors, beacons)]

In [6]:
lim = 4000000

# line segment from point11 to point12 has to have slope -1
# line segment from point21 to point22 has to have slope 1
# first line segment: (x, -x+point11[0]+point11[1])
# second line segment: (x, x-point21[0]+point21[1])
# intersection: (x, -x+point11[0]+point11[1]) == (x, x-point21[0]+point21[1])
# => 2x = point11[0]+point11[1]+point21[0]-point21[1]
# => x = (point11[0]+point11[1]+point21[0]-point21[1])/2
def intersection(point11, point12, point21, point22):
    x = (point11[0]+point11[1]+point21[0]-point21[1])/2
    y = -x+point11[0]+point11[1]
    if (max(0, point11[0], point21[0]) <= x <= min(lim, point12[0], point22[0])) and (0 <= y <= lim):
        return (x, y)
    return None

In [7]:
candidate_points = set()
intersection_points = set()
for sensor1, distance1 in zip(sensors, distances):
    for sensor2, distance2 in zip(sensors, distances):
        res = intersection((sensor1[0], sensor1[1]+distance1), (sensor1[0]+distance1, sensor1[1]), (sensor2[0], sensor2[1]-distance2), (sensor2[0]+distance2, sensor2[1]))
        if res is None:
            continue
        intersection_points.add(res)
        candidate_points.add((math.ceil(res[0])+1, math.floor(res[1])))
        candidate_points.add((math.ceil(res[0])+1, math.ceil(res[1])))
len(candidate_points)

84

In [8]:
found = None
for point in candidate_points:
    flag = True
    for sensor, distance in zip(sensors, distances):
        if dist(point, sensor) <= distance:
            flag = False
            break
    if flag:
        found = point
        break
found

(2949122, 3041245)

In [9]:
lim*found[0]+found[1]

11796491041245